<a href="https://colab.research.google.com/github/Itsuki-Hamano123/dim-compression/blob/master/experiments/XGBoost/compress_dim_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA(主成分分析)で次元圧縮

In [2]:
import os
import datetime

import cloudpickle
import pandas as pd

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb

In [3]:
print('{lib}:{ver}'.format(lib='xgboost',ver=xgb.__version__))
!pip show scikit-learn

xgboost:0.90
Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: joblib, scipy, numpy
Required-by: yellowbrick, umap-learn, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


# covtypeデータ読み込み
特徴量54次元で7クラス分類を行うデータ<br>
[scikit-learnのリファレンス](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype)<br>
[データセットの詳細ページ](https://archive.ics.uci.edu/ml/datasets/Covertype)


In [4]:
SEED = 2

data_bunch = fetch_covtype(random_state=SEED,
                           shuffle=True,
                           return_X_y=False)

print('data shape:{shape}'.format(shape=data_bunch.data.shape))
print('target shape:{shape}'.format(shape=data_bunch.target.shape))
data_bunch

data shape:(581012, 54)
target shape:(581012,)


{'DESCR': ".. _covtype_dataset:\n\nForest covertypes\n-----------------\n\nThe samples in this dataset correspond to 30×30m patches of forest in the US,\ncollected for the task of predicting each patch's cover type,\ni.e. the dominant species of tree.\nThere are seven covertypes, making this a multiclass classification problem.\nEach sample has 54 features, described on the\n`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.\nSome of the features are boolean indicators,\nwhile others are discrete or continuous measurements.\n\n**Data Set Characteristics:**\n\n    =================   ============\n    Classes                        7\n    Samples total             581012\n    Dimensionality                54\n    Features                     int\n    =================   ============\n\n:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;\nit returns a dictionary-like object\nwith the feature matrix in the ``data`` member\nand the target values

In [5]:
# 訓練データ,テストデータに分割
train_X, test_X, train_y, test_y = train_test_split(data_bunch.data, data_bunch.target,
                                                    test_size=0.2, random_state=SEED)
# 訓練データ,検証データに分割
train_X, validation_X, train_y, validation_y = train_test_split(train_X, train_y,
                                                                test_size=0.2, random_state=SEED)

print('train size:{size}'.format(size=train_y.shape[0]))
print('validation size:{size}'.format(size=validation_y.shape[0]))
print('test size:{size}'.format(size=test_y.shape[0]))

train size:371847
validation size:92962
test size:116203


In [28]:
NUM_CLASS = 7

# データの前処理

## 標準化

In [8]:
STD_SCALER = StandardScaler()
# 標準化器を作成
STD_SCALER.fit(train_X)
print('sclaer fit end')

sclaer fit end


In [9]:
# 訓練データを標準化
std_train_X = STD_SCALER.transform(train_X, copy=True)
std_train_X[:,0]

array([ 0.31952626, -1.6259504 ,  1.09557117, ..., -0.64605958,
       -0.18830037,  1.2529259 ])

In [10]:
# 検証/テストデータも標準化
std_validation_X = STD_SCALER.transform(validation_X, copy=True)
std_test_X = STD_SCALER.transform(test_X, copy=True)

## PCAで次元圧縮

In [11]:
# 累積寄与率
ACCUMULATION_CONTIBUTION_RATE = 0.90

pca = PCA(n_components=ACCUMULATION_CONTIBUTION_RATE)
pca.fit(std_train_X)

print('圧縮前の次元サイズ:{size}'.format(size=std_train_X.shape[1]))
print('圧縮後の次元サイズ:{size}'.format(size=len(pca.explained_variance_ratio_)))
print('累積寄与率:{rate}'.format(rate=pca.explained_variance_ratio_.sum()))

圧縮前の次元サイズ:54
圧縮後の次元サイズ:40
累積寄与率:0.9083114617566342


In [12]:
# 訓練/検証/テストデータをPCAで次元圧縮
pca_train_X = pca.transform(std_train_X)
pca_validation_X = pca.transform(std_validation_X)
pca_test_X = pca.transform(std_test_X)

# XGBosot(classifier)でクラス分類モデル構築

In [13]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [14]:
cd My Drive/機械学習練習/次元圧縮手法_比較/experiment/XGBoost

/gdrive/My Drive/機械学習練習/次元圧縮手法_比較/experiment/XGBoost


In [15]:
!ls 

compress_dim_ica.ipynb	compress_dim_nmf.ipynb	compress_dim_pca.ipynb


In [23]:
LOG_DIR = '../logs'
model_name = 'pca_xgboost'
xgb.XGBClassifier()
xgb_cls = xgb.XGBRFClassifier(random_state=SEED, tree_method='gpu_hist', gpu_id=0)

## モデルの訓練

In [24]:
xgb_cls.fit(X=pca_train_X, y=train_y-1)

XGBRFClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=0.8,
                colsample_bytree=1, gamma=0, gpu_id=0, learning_rate=1,
                max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
                n_estimators=100, n_jobs=1, nthread=None,
                objective='multi:softprob', random_state=2, reg_alpha=0,
                reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                subsample=0.8, tree_method='gpu_hist', verbosity=1)

## モデルの評価

In [30]:
train_score = xgb_cls.score(X=pca_train_X, y=train_y-1)
test_score = xgb_cls.score(X=pca_test_X, y=test_y-1)
print('train score:{score}'.format(score=train_score))
print('test score:{score}'.format(score=test_score))

train score:0.6409948177610684
test score:0.6422983916077898


### 主な分類指標を示すレポート

In [31]:
predict_y = xgb_cls.predict(data=pca_test_X)

report = classification_report(y_true=test_y, y_pred=predict_y+1)
print(report)

              precision    recall  f1-score   support

           1       0.64      0.48      0.55     42488
           2       0.64      0.83      0.73     56473
           3       0.62      0.86      0.72      7355
           4       0.62      0.25      0.36       524
           5       0.82      0.07      0.13      1866
           6       0.34      0.01      0.02      3432
           7       0.88      0.14      0.24      4065

    accuracy                           0.64    116203
   macro avg       0.65      0.38      0.39    116203
weighted avg       0.64      0.64      0.61    116203



### 混同行列

In [32]:
cm = confusion_matrix(y_true=test_y, y_pred=predict_y+1)
print(cm)

[[20455 21911    43     0     4     0    75]
 [ 8048 46968  1421     4    25     5     2]
 [    9   895  6351    55     0    45     0]
 [    7     9   370   131     0     7     0]
 [  394  1200   141     0   131     0     0]
 [   20  1385  1974    23     0    30     0]
 [ 2959   518    17     0     0     0   571]]


### モデルの保存

In [33]:
model_file = os.path.join(LOG_DIR, model_name+'_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.pkl')

with open(model_file, 'wb') as f:
      f.write(cloudpickle.dumps(xgb_cls))

In [34]:
!ls ../logs/

compress_dim_ica_20200706-000503.h5  fit
compress_dim_pca_20200705-080246.h5  ica_xgboost_20200712-055216.pkl
compress_dim_pca_20200705-080838.h5  ica_xgboost.pkl
compress_dim_pca_20200706-122832.h5  pca_xgboost_20200712-055438.pkl
